In [2]:
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from entity_list import ENTITIES

In [3]:
id_images = [] 
text_images = [] 
path_images = [] 

In [4]:
opt = Options()
opt.add_argument("--disable-infobars")
opt.add_argument("start-maximized")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-notifications")
opt.add_argument("--disable-gpu")
# opt.add_argument("--headless")
opt.add_experimental_option("prefs", { \
    "profile.default_content_setting_values.media_stream_mic": 1,       
    "profile.default_content_setting_values.media_stream_camera": 1,
    "profile.default_content_setting_values.geolocation": 1, 
    "profile.default_content_setting_values.notifications": 1 
  })

In [5]:
def generate_url(entity, idx_page): 
    return f'https://www.shutterstock.com/vi/search/{entity}?page={idx_page}'.replace(' ', '-')

def create_driver(): 
    driver = webdriver.Chrome(options = opt)

    return driver

def new_tab(driver, url): 
    driver.execute_script(f"window.open('{url}', '_blank')")

    return driver

def switch_tab(driver, idx_tag): 
    driver.switch_to.window(driver.window_handles[idx_tag])

    return driver

def close_tab(driver): 
    driver.close()
    driver = switch_tab(driver, idx_tag=0)

    return driver

def collect_image(url, id_image): 
    response = requests.get(url)
    path = f"images/{id_image}.jpg"
    
    with open(path, 'wb') as f:
        f.write(response.content)

    return

In [6]:
max_iter = 2 # set number of page crawl iterations

In [7]:
try: 
    driver = create_driver()
    count = 0
    
    for entity in ENTITIES: 
        # for idx_page in range(1, max_iter+1):
        for idx_page in range(16, 17):
            url = generate_url(entity=entity
                            , idx_page=idx_page)
            
            driver.get(url)
            elements = driver.find_elements(By.XPATH, "//div[@role='img']")
            
            for element in elements:
                url_item = element.find_element(By.TAG_NAME, 'a').get_attribute('href')

                driver = new_tab(driver, url_item)
                driver = switch_tab(driver, 1)
                
                # check invalid item (removed by website)
                try:           
                    driver.find_element(By.CLASS_NAME, 'MuiAlert-message.mui-1xsto0d')
                    print('skipp')
                    driver = close_tab(driver)
                    continue
                except: 
                    pass            
                
                
                url_image = driver.find_element(By.CLASS_NAME, 'mui-1jn9gxg-link-disabled')\
                    .find_element(By.TAG_NAME, 'picture').find_elements(By.TAG_NAME, 'source')[-1].get_attribute('srcset')
                
                collect_image(url_image, count)
                
                text = driver.find_element(By.CLASS_NAME, 'MuiTypography-root.MuiTypography-body1.mui-tw73me-bold-title').text
                
                driver = close_tab(driver) 
                
                id_images.append(count)
                text_images.append(text)
                path_images.append(f'images/{count}.jpg')
                
                count += 1
                
                # scroll
                scroll_value = 500
                script = f"window.scrollBy(0, {scroll_value});"
                driver.execute_script(script)
                
    driver.quit()

except Exception as e: 
    pass

In [8]:
df = pd.DataFrame({'id' : id_images, 
                  'caption' : text_images, 
                  'link_img' : path_images})

df.to_csv('results.csv', index=False)